In [6]:
# self-define function
from data_cleaning import cleaning_text, sentence_df
from participant_function import participants_list, clean_participants_list
from stock_price_info import stock_price_change

# read and saving path
path = "/Users/timliu/Documents/GitHub/final_01/Transcript_test"
# path = "/Users/timliu/Documents/GitHub/final_01/Tim_test"
save_path = "/Users/timliu/Documents/GitHub/final_01/Output"

# path = "/Users/hienanh/Documents/GitHub/final_01/Transcript_test"
# save_path = "/Users/hienanh/Documents/GitHub/final_01/Output"

# other packages
import pdftotext
import pandas as pd
import numpy as np
import re
import os
import string
# from collections import Counter
# from sklearn.model_selection import train_test_split
# from sklearn.pipeline import Pipeline
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
# from sklearn.metrics import confusion_matrix,classification_report, accuracy_score
# from sklearn.linear_model import LogisticRegression

import nltk 
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples
from nltk.corpus import stopwords          # module for stop words that come with NLTK
nltk.download('stopwords')
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings

import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/hienanh/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hienanh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# all files path
company_paths = []
sectors = os.listdir(path) 
if '.DS_Store' in sectors:
    sectors.remove('.DS_Store')

for sector in sectors:
    # path to each sector files
    sector_path = path+"/"+sector
    
    # path to each company files
    companies = os.listdir(sector_path)
    if '.DS_Store' in companies:
        companies.remove('.DS_Store')
    
    for company in companies:
        tmp = sector_path+"/"+company
        company_paths.append(tmp)

In [3]:
# # checking with DF (not necessary)
# company_paticipants_df = pd.DataFrame()
# other_paticipants_df = pd.DataFrame()

df_clean_na = pd.DataFrame(np.zeros((2500,1)), columns=['index']) # create a dataframe with 2500 rows
# all_participants = []      
company_paticipants_list = [] 
other_paticipants_list = []

for single_path in company_paths:
    files = os.listdir(single_path)
    for file in files:
        # print(file)
        if file.endswith(".pdf"):
            # print(file)
            # Load PDF
            with open(single_path+"/"+file, "rb") as f:
                pdf = pdftotext.PDF(f)
            # Save all text to a txt file.
            with open(save_path+"/"+file.replace(".pdf", ".txt"), "w") as f:
                f.write("\n\n".join(pdf))
            # open the text file
            with open(save_path+"/"+file.replace(".pdf", ".txt")) as f:
                contents = f.readlines()
                df_clean = cleaning_text(contents)    ##### here we use the function from data_cleaning.py
                
                index_presentation = df_clean[df_clean[0] == 'Presentation'].index
                if len(index_presentation) == 2:
                    df_clean.loc[index_presentation[0], 0] = 'heading' # replace the first index_presentation in df_clean with "head" # interacting with the participant_function.py

                # extract the participants
                tmp_df_text,tmp_company_paticipants,tmp_other_paticipants = participants_list(df_clean) ##### here we use the function from participant_function.py
                company_paticipants_list.append(tmp_company_paticipants)
                other_paticipants_list.append(tmp_other_paticipants)
                

                # # checking with DF (not necessary)
                # company_paticipants = pd.DataFrame(company_paticipants)
                # if company_paticipants.empty:
                #     company_paticipants = pd.DataFrame(np.zeros((1,1)), columns=['index'])
                # company_paticipants_df[f"{files[files.index(file)]}"] = company_paticipants
                # other_paticipants = pd.DataFrame(other_paticipants)
                # if other_paticipants.empty:
                #     other_paticipants = pd.DataFrame(np.zeros((1,1)), columns=['index'])
                # other_paticipants_df[f"{files[files.index(file)]}"] = other_paticipants
                
                df = pd.DataFrame()
                df[f"{files[files.index(file)]}"] = tmp_df_text
                # remove the row using re, if the row ends with Presentation  Investor Day	
                df_clean_na[f"{files[files.index(file)]}"] = df[f"{files[files.index(file)]}"].dropna(inplace=False).reset_index(drop=True)

# drop the first column of the df
df_clean_na = df_clean_na.iloc[:,1:]
df_clean_na.head(5)


/var/folders/sq/sxhsy98j6f57m406rd5lz1_r0000gn/T/ipykernel_31209/3062513208.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_clean_na[f"{files[files.index(file)]}"] = df[f"{files[files.index(file)]}"].dropna(inplace=False).reset_index(drop=True)


,20121119_Sompo_Holdings_Inc-_Earnings_Call_2012-11-20_RT000000002067692926.pdf,20191128_Sompo_Holdings_Inc-_Conf-Presentation_Call_2019-11-28_SD000000002903087817.pdf,20180524_Sompo_Holdings_Inc-_M-A_Call_2018-5-24_FS000000002476688929.pdf,20190520_Sompo_Holdings_Inc-_Earnings_Call_2019-5-20_SD000000002902929962.pdf,20120518_Sompo_Holdings_Inc-_Earnings_Call_2012-5-21_RT000000002050742732.pdf,20141119_Sompo_Holdings_Inc-_Earnings_Call_2014-11-24_FS000000002417374854.pdf,20151118_Sompo_Holdings_Inc-_Earnings_Call_2015-11-20_DN000000002241138647.pdf,20131119_Sompo_Holdings_Inc-_Earnings_Call_2013-11-25_DN000000002116348219.pdf,20181119_Sompo_Holdings_Inc-_Earnings_Call_2018-11-19_SD000000002905610039.pdf,20151126_Sompo_Holdings_Inc-_Conf-Presentation_Call_2015-11-26_SD000000002889371132.pdf,...,20210810_Munich_Re-_Earnings_Call_2021-8-10_DN000000002962247258.pdf,20201208_Munich_Re-_Shareholder_Mtg_Call_2020-12-8_DN000000002944723654.pdf,20200507_Munich_Re-_Earnings_Call_2020-5-7_RT000000002834670399.pdf,20171109_Munich_Re-_Earnings_Call_2017-11-9_FS000000002383615110.pdf,20170509_Munich_Re-_Earnings_Call_2017-5-9_FS000000002450443745.pdf,20170315_Munich_Re-_Earnings_Call_2017-3-15_FS000000002336461963.pdf,20151105_Munich_Re-_Earnings_Call_2015-11-5_FS000000002238544235.pdf,20171121_Munich_Re-_Shareholder_Mtg_Call_2017-11-21_SD000000002905542974.pdf,20190320_Munich_Re-_Earnings_Call_2019-3-27_RT000000002898185222.pdf,20150806_Munich_Re-_Earnings_Call_2015-8-6_SD000000002845566471.pdf
0,Shinichi Hara,Kengo Sakurada,Unverified Participant,Osamu Nose,Shinichi Hara,Yukinori Kuroda,Yukinori Kuroda,Shinichi Hara,Yukinori Kuroda,Kengo Sakurada,...,Operator,"Good afternoon, ladies and gentlemen and warm ...",Operator,Christian Becker-Hussong,Operator,Christian Becker-Hussong,Operator,Christian Becker-Hussong,Christian Becker-Hussong,Joerg Schneider
1,I am Hara from NKSJ Holdings. Thank you very m...,My name is Sakurada from Sompo Holdings. Thank...,I thank all of you for gathering here today fo...,"Hello. I am Osamu Nose, Head of Investor Relat...",Good evening. This is Shinichi Hara of NKSJ Ho...,"Hello, everyone. I'm Kuroda of Investor Relati...",So this is Kuroda from Sompo Japan Nipponkoa H...,I'm Hara from NK Share Holdings. Thank you ver...,I'm Kuroda with Sompo Holdings. Thank you very...,"As introduced, I am Sakurada, the Group CEO. F...",...,"Good day, and welcome to the Munich Re Half Ye...",very -- we very much hope you are all well and...,"Good day, and welcome to the Muenchener Quarte...",Thanks for joining us for our Q3 earnings call...,"Good day, ladies and gentlemen, and welcome to...","Ladies and gentlemen, welcome to the Munich Re...","Good day, ladies and gentlemen. Welcome to the...",transformation. Welcome to everybody here in t...,Thank you. Ladies and gentlemen a warm welcome...,"Ladies and gentlemen, welcome to the presentat..."
2,sure you're tired by now at this late hour but...,expense of your very busy schedule.,are as we have announced on the 18th. Therefor...,Holdings. Thank you for participating in our c...,busy schedule to attend this conference call. ...,Thank you very much for joining us today in th...,participating in today's conference call.,"call for the first half results. Today, I'll c...",schedule to this conference call. I would like...,meeting despite your busy schedule. It seems t...,...,Conference Call. Today's conference is being r...,"Before we start, please allow us a few houseke...",Results Conference Call. Today's conference is...,"very straightforward, after my short introduct...",Q1 Conference Call. Today's conference is bein...,room. And of course also a very warm welcome t...,Statement Conference Call. Today's conference ...,everybody following this conference on the Int...,Financial Results 2018 and Strategic Outlook. ...,Schneider and I am the CFO of Munich Re Group.
3,What I am going to explain today is only the f...,We just made the announcement of the results o...,"for myself toda

In [4]:
# generating all the participants
company_paticipants_list = clean_participants_list(company_paticipants_list)
other_paticipants_list = clean_participants_list(other_paticipants_list)
all_participants_list_tmp = []
all_participants_list_tmp = company_paticipants_list + other_paticipants_list

all_participants_list = []
for x in all_participants_list_tmp:
    if x not in all_participants_list:
        all_participants_list.append(x)


In [5]:
concat_df = pd.DataFrame()
for column in df_clean_na.columns:
    # identify all the rows in df with all_participants_list in it
    both_participants_row_index = df_clean_na[df_clean_na[column].isin(all_participants_list)].index.tolist()
    # apply the both_participants_row_index to the df_clean_na['participants']
    concat_df[column] = df_clean_na[column]
    concat_df[f"participants_{column}"] = df_clean_na[column].apply(lambda x: x if x in all_participants_list else np.nan)
    # fill the NaN with the value of the previous row
    concat_df[f"participants_{column}"] = concat_df[f"participants_{column}"].fillna(method='ffill')

concat_df

/var/folders/sq/sxhsy98j6f57m406rd5lz1_r0000gn/T/ipykernel_31209/720997122.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  concat_df[column] = df_clean_na[column]
/var/folders/sq/sxhsy98j6f57m406rd5lz1_r0000gn/T/ipykernel_31209/720997122.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  concat_df[f"participants_{column}"] = df_clean_na[column].apply(lambda x: x if x in all_participants_list else np.nan)


,20121119_Sompo_Holdings_Inc-_Earnings_Call_2012-11-20_RT000000002067692926.pdf,participants_20121119_Sompo_Holdings_Inc-_Earnings_Call_2012-11-20_RT000000002067692926.pdf,20191128_Sompo_Holdings_Inc-_Conf-Presentation_Call_2019-11-28_SD000000002903087817.pdf,participants_20191128_Sompo_Holdings_Inc-_Conf-Presentation_Call_2019-11-28_SD000000002903087817.pdf,20180524_Sompo_Holdings_Inc-_M-A_Call_2018-5-24_FS000000002476688929.pdf,participants_20180524_Sompo_Holdings_Inc-_M-A_Call_2018-5-24_FS000000002476688929.pdf,20190520_Sompo_Holdings_Inc-_Earnings_Call_2019-5-20_SD000000002902929962.pdf,participants_20190520_Sompo_Holdings_Inc-_Earnings_Call_2019-5-20_SD000000002902929962.pdf,20120518_Sompo_Holdings_Inc-_Earnings_Call_2012-5-21_RT000000002050742732.pdf,participants_20120518_Sompo_Holdings_Inc-_Earnings_Call_2012-5-21_RT000000002050742732.pdf,...,20170315_Munich_Re-_Earnings_Call_2017-3-15_FS000000002336461963.pdf,participants_20170315_Munich_Re-_Earnings_Call_2017-3-15_FS000000002336461963.pdf,20151105_Munich_Re-_Earnings_Call_2015-11-5_FS000000002238544235.pdf,participants_20151105_Munich_Re-_Earnings_Call_2015-11-5_FS000000002238544235.pdf,20171121_Munich_Re-_Shareholder_Mtg_Call_2017-11-21_SD000000002905542974.pdf,participants_20171121_Munich_Re-_Shareholder_Mtg_Call_2017-11-21_SD000000002905542974.pdf,20190320_Munich_Re-_Earnings_Call_2019-3-27_RT000000002898185222.pdf,participants_20190320_Munich_Re-_Earnings_Call_2019-3-27_RT000000002898185222.pdf,20150806_Munich_Re-_Earnings_Call_2015-8-6_SD000000002845566471.pdf,participants_20150806_Munich_Re-_Earnings_Call_2015-8-6_SD000000002845566471.pdf
0,Shinichi Hara,Shinichi Hara,Kengo Sakurada,Kengo Sakurada,Unverified Participant,Unverified Participant,Osamu Nose,Osamu Nose,Shinichi Hara,Shinichi Hara,...,Christian Becker-Hussong,Christian Becker-Hussong,Operator,Operator,Christian Becker-Hussong,Christian Becker-Hussong,Christian Becker-Hussong,Christian Becker-Hussong,Joerg Schneider,Joerg Schneider
1,I am Hara from NKSJ Holdings. Thank you very m...,Shinichi Hara,My name is Sakurada from Sompo Holdings. Thank...,Kengo Sakurada,I thank all of you for gathering here today fo...,Unverified Participant,"Hello. I am Osamu Nose, Head of Investor Relat...",Osamu Nose,Good evening. This is Shinichi Hara of NKSJ Ho...,Shinichi Hara,...,"Ladies and gentlemen, welcome to the Munich Re...",Christian Becker-Hussong,"Good day, ladies and gentlemen. Welcome to the...",Operator,transformation. Welcome to everybody here in t...,Christian Becker-Hussong,Thank you. Ladies and gentlemen a warm welcome...,Christian Becker-Hussong,"Ladies and gentlemen, welcome to the presentat...",Joerg Schneider
2,sure you're tired by now at this late hour but...,Shinichi Hara,expense of your very busy schedule.,Kengo Sakurada,are as we have announced on the 18th. Therefor...,Unverified Participant,Holdings. Thank you for participating in our c...,Osamu Nose,busy schedule to attend this conference call. ...,Shinichi Hara,...,room. And of course also a very warm welcome t...,Christian Becker-Hussong,Statement Conference Call. Today's conference ...,Operator,everybody following this conference on the Int...,Christian Becker-Hussong,Financial Results 2018 and Strategic Outlook. ...,Christian Becker-Hussong,Schneider and I am the CFO of Munich Re Group.,Joerg Schneider
3,What I am going to explain today is only the f...,Shinichi Hara,We just made the announcement of the results o...,Kengo Sakurada,"for myself today, I will be mainly talking abo...",Unverified Participant,I would like to go through our business result...,Osamu Nose,presentation on Fiscal 2011 Results and Foreca...,Shinichi Hara,...,from their offices via Internet.,Christian Becker-Hussong,"At this time, I would like to turn the confere...",Operator,We are delighted to have 11 colleagues with us...,Christian Becker-Hussong,we get started I would like to express some ou...,Christian Becker-Hussong,"As usual, I will provide you with the main mes.

In [17]:
model_df =  sentence_df(df_clean_na, concat_df, company_paticipants_list, other_paticipants_list)
# model_df grpupby and join if 'participants','file_name','date', 'company_paticipants_yes' are all the same
test = model_df.groupby(['participants','file_name','date', 'company_paticipants_yes']).agg('sum').reset_index()
test

,participants,file_name,date,company_paticipants_yes,other_paticipants_yes
0,Corporate Participant,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,1,0
1,Futoshi Sasaki,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,0,24
2,Jun Shiota,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,0,14
3,Kazuharu Miura,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,0,13
4,Masao Muraki,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,0,18
5,Mitsumasa Okamoto,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,0,13
6,Natsumu Tsujino,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,0,20
7,Operator,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,1,16
8,Shinichi Hara,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,1,0
9,Wataru Otsuka,20121119_Sompo_Holdings_Inc-_Earnings_Call_201...,2012-11-19,0,13


In [14]:
model_df['sentence'].loc[1]

'I am Hara from NKSJ Holdings. Thank you very much for attending our meeting today. I am'

In [15]:
model_df['sentence'].loc[2]

"sure you're tired by now at this late hour but please bear with us for a little bit longer."

In [16]:
model_df['sentence'].loc[3]

'What I am going to explain today is only the first half results, the business forecast for the'

In [81]:
# dataframe company and its tickers 這邊我們只取兩個categories的資料
stock_list = [
    # European (Re)Insurers
    'SCR.PA','MUV2.DE','SREN.SW','HNR1.DE',
    # Japanese & Pacific
    '8766.T','8766.T','8766.T', # this company has 3 file name
    '8630.T','8725.T','QBE.AX',
]
company_name = [
    # European (Re)Insurers
    'SCOR SE-','Munich Re-','Swiss Re','Hannover Rueck',
    # Japanese & Pacific
    'TOKIO MARINE', 'HCC INSURANCE-','MITSUI FUDOSAN-',
    'Sompo Holdings','MS-AD INSURANCE-','QBE INSURANCE-'
               ]
company_ticker_df = pd.DataFrame({'company_name':company_name,
                                  'ticker':stock_list})

company_ticker_df

,company_name,ticker
0,SCOR SE-,SCR.PA
1,Munich Re-,MUV2.DE
2,Swiss Re,SREN.SW
3,Hannover Rueck,HNR1.DE
4,TOKIO MARINE,8766.T
5,HCC INSURANCE-,8766.T
6,MITSUI FUDOSAN-,8766.T
7,Sompo Holdings,8630.T
8,MS-AD INSURANCE-,8725.T
9,QBE INSURANCE-,QBE.AX


In [85]:
stock_price_df = stock_price_change(stock_list, '2010-12-01', '2022-01-30')  ##### here we use the function from stock_price_change.py

# merged company tickers into big dataframe
prc_n_tkr_df = model_df.copy()
prc_n_tkr_df = prc_n_tkr_df.join(company_ticker_df.set_index('company_name'), 
                         on='company_name',
                         how='left'
                        )
#print(len(prc_n_tkr_df))
# merged stock price in
#merged_data= prc_n_tkr_df.merge(stock_price_df, on=["date","ticker"], how='left')
merged_data = prc_n_tkr_df.join(stock_price_df.set_index(["date","ticker"]), 
                         on=["date","ticker"],
                         how='left'
                        )
# drop na rows, which date in on Sat or Sun so it did not have the stock price
# merged_data.loc[merged_data['Close'].isnull()==True]
#merged_data = merged_data.dropna()
merged_data = merged_data.reset_index(drop=True)
merged_data

,date,Close,ticker,D-1,D+1,D-2,D+2,D-3,D+3,D-5,D+5,D-10,D+10,D-15,D+15
0,2010-12-01,18.190001,SCR.PA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,2010-12-02,18.309999,SCR.PA,0.006597,-0.004369,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,2010-12-03,18.230000,SCR.PA,-0.004369,-0.000549,0.002199,0.009600,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,2010-12-06,18.219999,SCR.PA,-0.000549,0.010154,-0.004915,0.018386,0.001649,0.033480,0.0,0.0,0.0,0.0,0.0,0.0
4,2010-12-07,18.405001,SCR.PA,0.010154,0.008150,0.009600,0.023092,0.005188,0.020103,0.0,0.0,0.0,0.0,0.0,0.0
